In [1]:
from functions import *
from ipywidgets import HBox, HTML, Dropdown, Label, Layout,Image
import tkinter as tk
from IPython.display import display, Markdown, clear_output, IFrame,Javascript
from pyvis.network import Network
pathways_name=pd.read_csv("data/pathways.tsv", sep='\t')["pathway_name"]
last_selection=pathways_name
display(HTML("<style>.container { width:100% !important; }</style>"))


HTML(value='<style>.container { width:100% !important; }</style>')

In [2]:
display(HTML("<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>"))
text=Label(value="")
text.add_class("left-spacing-class")
menu = Dropdown(
       options=list(pathways_name.values),
       value=list(pathways_name.values)[25],
       description='Pathway:')
sidebyside=HBox([menu, HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells([2])"><input type="submit" id="toggleButton" value="Run"></form>'''),text])

display(sidebyside)

HTML(value='<style>.left-spacing-class {margin-left: 20px; color:blac; margin-right:20px}</style>')

In [4]:
skip_calcs=False
selection=menu.value
pathway_edges=read_pathway(selection)
if (len(pathway_edges)==0):
    selection=last_selection
    skip_calcs=True
    st.error("Edges not found, try another pathway!")
else:
    skip_calcs=False
    last_selection=selection
    pathway_edges=read_pathway(selection)
    adj_matrix,nodes_renamed,inv_nodes_renamed=build_adj(pathway_edges)
    G = nx.from_numpy_matrix(adj_matrix)
    triad_cliques=get_triad(G)
    weighted_edges=calculate_weighted_edges(triad_cliques, adj_matrix,inv_nodes_renamed)
    to_remove=[]
    signify_values={}
    essential_edges=[]
    for x in weighted_edges.items():
        zeros=0
        ones=0
        minus=0
        for z in x[1]:
            if (z[1]==0):
                zeros+=1
            elif (z[1]==1):
                ones+=1
            else:
                minus+=1
        if (ones==0):
            if (minus==0):
                to_remove.append(x[0])
            else:
                m=(zeros+minus)/2
                if ((minus+zeros)/(zeros*minus+1)*zeros/(minus+zeros)>((minus+zeros)/(m*m+1))*zeros/(minus+zeros)):
                    to_remove.append(x[0])
        else:
            essential_edges.append(x[0])
        if (ones==0):
            signify_values[x[0]]=(minus+zeros)/(zeros*minus+1)*zeros/(minus+zeros)
        else:
            signify_values[x[0]]=0

    relabel={}
    for e,node in enumerate( G.nodes()):
        relabel[e]=str(inv_nodes_renamed[node])
net=Network(height="825px",notebook=True,directed=True,width="1800px", bgcolor='#222222', font_color='white')
for i,node in relabel.items():
    net.add_node(str(node))

for edge in pathway_edges.values:
        if(edge[2]==-1):
            net.add_edge(str(edge[0]), str(edge[1]), color="yellow")
        else:
            net.add_edge(str(edge[0]), str(edge[1]))
for triad in triad_cliques:
    for i,x in enumerate(triad):
        for j,y in enumerate(triad):
            value=""
            isessential=""
            tmp=pathway_edges[(pathway_edges[0]==inv_nodes_renamed[triad[i]]) & (pathway_edges[1]==inv_nodes_renamed[triad[j]])].values
            if (len(tmp)>0):
                start_node,to_node,weight=tmp[0]
            else:
                continue
            if ((str(start_node)+","+str(to_node)) in to_remove):           
                color="red"
                size=10
                value+=", significativity:  "+str(signify_values[str(start_node)+","+str(to_node)])
            else:
                color="green"
                size=3
                value+=", significativity:  "+str(signify_values[str(start_node)+","+str(to_node)])
            if ((str(start_node)+","+str(to_node)) in essential_edges):   
                isessential="Essential "
            if (weight==1):
                net.add_edge(str(start_node), str(to_node), color=color, width=size,title=isessential+"Expression edge"+value)
            else:
                net.add_edge(str(start_node), str(to_node), color=color, width=size,title=isessential+"Suppression edge"+value)
net.hrepulsion(node_distance=120, central_gravity=0.0, spring_length=100, spring_strength=0, damping=0.09)
net.show("data/graph.html")

In [19]:
signify_values

{'6256,4023': 0,
 '51129,4023': 0.2222222222222222,
 '6256,51129': 1.0,
 '6258,4023': 0.0,
 '6258,51129': 0.0,
 '6257,4023': 0.0,
 '6257,51129': 0.0,
 '5465,4023': 0.0,
 '5465,51129': 0.0,
 '5467,4023': 0.0,
 '5467,51129': 0.0,
 '5468,4023': 0,
 '5468,51129': 1.0}

In [25]:
weighted_edges

{'6256,4023': [(0.751351887950401, 1)],
 '51129,4023': [(0.00958993913913588, 0),
  (0.42793669112729465, -1),
  (0.493972998423884, -1),
  (0.4154314035435193, -1),
  (0.47398475209831153, -1),
  (-0.08148364214020982, 0)],
 '6256,51129': [(0.00958993913913588, 0)],
 '6258,4023': [(0.31271902702610976, -1)],
 '6258,51129': [(0.42793669112729465, -1)],
 '6257,4023': [(-0.18965576363770847, -1)],
 '6257,51129': [(0.493972998423884, -1)],
 '5465,4023': [(0.5002211698648119, -1)],
 '5465,51129': [(0.4154314035435193, -1)],
 '5467,4023': [(-0.11442379562472764, -1)],
 '5467,51129': [(0.47398475209831153, -1)],
 '5468,4023': [(0.99691599298314, 1)],
 '5468,51129': [(-0.08148364214020982, 0)]}

In [64]:
weighted_edges={}
first_label=""
second_label=""
third_label=""
mod = """ 
          y ~ x1 + x2
          """
for triad in range(len(triad_cliques)):
    triad_matrix=np.zeros((3,3))
    for i,x in enumerate(triad_cliques[triad]):
        for j,y in enumerate(triad_cliques[triad]):
            triad_matrix[i][j]=adj_matrix[x][y]
  #  if list(triad_matrix[0])== [0, 1, 1]:
    print(triad_matrix)
    zeros_count=np.array([len(np.where(x==0)[0]) for i,x in enumerate(triad_matrix) ])
    first_index=int(np.where(zeros_count==1)[0])
    second_index=int(np.where(zeros_count==2)[0])
    third_index=int(np.where(zeros_count==3)[0])
    first_label=str(inv_nodes_renamed[triad_cliques[triad][first_index]])
    second_label=str(inv_nodes_renamed[triad_cliques[triad][second_index]])
    third_label=str(inv_nodes_renamed[triad_cliques[triad][third_index]])
    first_gene=(list(esets.loc[first_label,:].values),0)
    second_gene=(list(esets.loc[second_label,:].values),1)
    third_gene=(list(esets.loc[third_label,:].values),2)
    print(second_label)
    y=third_gene
    x1=first_gene
    x2=second_gene
    to_df={"y":y[0],"x1":x1[0],"x2":x2[0]}
    data=pd.DataFrame(to_df).replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()
    m = Model(mod)
    r = m.fit(data)
    fac_sum=np.abs(r.x[0]+r.x[1])
    if (np.abs(r.x[0])<fac_sum*0.1):
        if (first_label+","+third_label in weighted_edges):
            weighted_edges[first_label+","+third_label].append((r.x[0],0))

        else:
            weighted_edges[first_label+","+third_label]=[(r.x[0],0)]
        if (second_label+","+third_label in weighted_edges) :
            weighted_edges[second_label+","+third_label].append((r.x[1],1))
        else:
            weighted_edges[second_label+","+third_label]=[(r.x[1],1)]

        if(first_label+","+second_label in weighted_edges) :
            weighted_edges[first_label+","+second_label].append((r.x[1],1))
        else:
            weighted_edges[first_label+","+second_label]=[(r.x[1],1)]
    elif(np.abs(r.x[1])<fac_sum*0.1):
        if (first_label+","+third_label in weighted_edges):
            weighted_edges[first_label+","+third_label].append((r.x[0],1))

        else:
            weighted_edges[first_label+","+third_label]=[(r.x[0],1)]
        if (second_label+","+third_label in weighted_edges):  
            weighted_edges[second_label+","+third_label].append((r.x[1],0))
        else:
            weighted_edges[second_label+","+third_label]=[(r.x[1],0)]
        if(first_label+","+second_label in weighted_edges) :
            weighted_edges[first_label+","+second_label].append((r.x[1],0))
        else:
            weighted_edges[first_label+","+second_label]=[(r.x[1],0)]
    else:
        if (first_label+","+third_label in weighted_edges):
            weighted_edges[first_label+","+third_label].append((r.x[0],-1))

        else:
            weighted_edges[first_label+","+third_label]=[(r.x[0],-1)]
        if (second_label+","+third_label in weighted_edges):  
            weighted_edges[second_label+","+third_label].append((r.x[1],-1))
        else:
            weighted_edges[second_label+","+third_label]=[(r.x[1],-1)]
        if(first_label+","+second_label in weighted_edges) :
            weighted_edges[first_label+","+second_label].append((r.x[1],-1))
        else:
            weighted_edges[first_label+","+second_label]=[(r.x[1],-1)]

[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
57697
[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
2189
[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
2175
[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
57697
[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
2189
[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0. -1.  0.]]
2175


In [31]:
inv_nodes_renamed[26]

55215

In [54]:
triad_matrix

array([[ 0.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0., -1.,  0.]])

In [53]:
[np.where(x[i]==0) for i,x in enumerate() ]

(array([0, 1, 1, 1, 2, 2], dtype=int64),
 array([0, 0, 1, 2, 0, 2], dtype=int64))

In [58]:
[len(np.where(x==0)[0]) for i,x in enumerate(triad_matrix) ]

[1, 3, 2]

In [63]:
first=int(np.where(np.array([len(np.where(x==0)[0]) for i,x in enumerate(triad_matrix) ])==2)[0])
first

2

In [56]:
for x in triad_matrix:
    print(x)

[0. 1. 1.]
[0. 0. 0.]
[ 0. -1.  0.]


In [46]:
adj_matrix[27][26]

-1.0

In [27]:
nodes_renamed[4023]

26

In [45]:
pathway_edges_0=pathway_edges[0].unique()
pathway_edges_1=pathway_edges[1].unique()
nodes=list(np.hstack((pathway_edges_0,pathway_edges_1)))
nodes_renamed={}
inv_nodes_renamed={}
for e,x in enumerate(nodes):
    nodes_renamed[x]=e
    inv_nodes_renamed[e]=x
nodes=len(nodes)
adj_matrix=np.zeros((nodes,nodes))
for x in pathway_edges.values:
    print(x[0],x[1])
    adj_matrix[nodes_renamed[x[0]]][nodes_renamed[x[1]]]=x[2]

9894 545
84126 55215
84126 57697
84126 2189
84126 2175
7398 55120
91442 55215
91442 2177
79728 5889
545 672
545 55215
545 7157
545 57697
545 2189
545 1111
545 2175
545 11200
545 8878
675 5888
675 5889
378708 55215
378708 2177
201254 55215
201254 2177
57697 55215
57697 2177
22909 5395
22909 4292
57599 55120
55120 55215
55120 2177
3280 2177
3280 55215
83990 5395
83990 4292
100526739 55215
100526739 2177
2177 675
2177 22909
2176 55215
2176 2177
2175 55215
2175 2177
29089 55215
29089 2177
2188 55215
2188 2177
2187 55215
2187 2177
2178 55215
2178 2177
2189 55215
2189 2177
80233 55215
80233 2177


In [38]:
pathway_edges

,0,1,2
0,9894,545,1
1,84126,55215,1
2,84126,57697,1
3,84126,2189,1
4,84126,2175,1
5,7398,55120,-1
6,91442,55215,-1
7,91442,2177,-1
8,79728,5889,1
9,545,672,1
